In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json("Data/Yelp JSON/yelp_academic_dataset_business.json", lines=True)

bad_states = set(["AB", "HI", "UT", "SD", "XMS", "VI"]) # states either not in US, or manually checked the 1-2 businesses to be wrong
df = df[["business_id", "city", "state", "stars", "review_count"]]
df = df[~df["state"].isin(bad_states)]

df.head(3)

,business_id,city,state,stars,review_count
0,Pns2l4eNsfO8kk83dixA6A,Santa Barbara,CA,5.0,7
1,mpf3x-BjTdTEA3yCZrAYPw,Affton,MO,3.0,15
2,tUFrWirKiKi_TAnsVWINQQ,Tucson,AZ,3.5,22


In [3]:
import re

regex_rules = [
    (re.compile(r'\bst\.?\b'), "saint"),      # st. to saint
    (re.compile(r'\bmt\.?\b'), "mount"),      # mt. to mount
    (re.compile(r'[^a-z ]'), ''),             # remove all except letters, space
    (re.compile(r' {2,}'), ' ')               # combine multiple spaces into 1
]
def clean_city_name(city: str) -> str:
    city = city.strip().lower()
    for pattern, replacement in regex_rules:
        city = pattern.sub(replacement, city)
    return city.replace(" ","") # wanted to replace with _, but didnt work in places csv???

In [4]:
df["city"] = df["city"].apply(clean_city_name)
df["tup"] = df[["city", "state"]].apply(tuple, axis=1)
df.head()

,business_id,city,state,stars,review_count,tup
0,Pns2l4eNsfO8kk83dixA6A,santabarbara,CA,5.0,7,"(santabarbara, CA)"
1,mpf3x-BjTdTEA3yCZrAYPw,affton,MO,3.0,15,"(affton, MO)"
2,tUFrWirKiKi_TAnsVWINQQ,tucson,AZ,3.5,22,"(tucson, AZ)"
3,MTSW4McQd7CbVtyjqoe9mw,philadelphia,PA,4.0,80,"(philadelphia, PA)"
4,mWMc6_wTdE0EUBKIGXDVfA,greenlane,PA,4.5,13,"(greenlane, PA)"


In [5]:
place_df = pd.read_csv("./Data/places.csv")
place_df.head()

,City,State,StateCode,PlaceCode
0,tucson,AZ,4,77000
1,philadelphia,PA,42,60000
2,universitycity,MO,29,75220
3,lebanon,TN,47,41520
4,tampa,FL,12,8150


In [ ]:
# StateCode should be left zero padded to length 2, PlaceCode should be left zero padded to length 5 for Census API
def formatStateCode(code: int | str) -> str:
    code = str(code)
    return code.zfill(2)
def formatPlaceCode(placeCode: int | str)->str:
    placeCode = str(placeCode)
    return placeCode.zfill(5)

place_df["StateCode"] = place_df["StateCode"].apply(formatStateCode)
place_df["PlaceCode"] = place_df["PlaceCode"].apply(formatPlaceCode)

In [6]:
clean_df = place_df.drop_duplicates().copy()
clean_df.to_csv("places_clean.csv")

In [7]:
clean_df["tup"] = clean_df[["City", "State"]].apply(tuple, axis=1)
clean_df.head()

,City,State,StateCode,PlaceCode,tup
0,tucson,AZ,4,77000,"(tucson, AZ)"
1,philadelphia,PA,42,60000,"(philadelphia, PA)"
2,universitycity,MO,29,75220,"(universitycity, MO)"
3,lebanon,TN,47,41520,"(lebanon, TN)"
4,tampa,FL,12,8150,"(tampa, FL)"


In [17]:
place_loc_set = set(clean_df["tup"])

missing_locs = df.loc[~df["tup"].isin(place_loc_set), "tup"]#.unique()

In [12]:
print(f"{missing_locs.shape[0]} Missing Locations")
print(missing_locs)

371 Missing Locations
[('greenlane', 'PA') ('fernley', 'NV') ('warrington', 'PA')
 ('terraceia', 'FL') ('delawarecounty', 'PA') ('valleyforge', 'PA')
 ('mapleshade', 'NJ') ('cedarbrook', 'NJ') ('perkiomenville', 'PA')
 ('view', 'TN') ('buckingham', 'PA') ('riverside', 'NJ') ('delran', 'NJ')
 ('limerick', 'PA') ('cinnaminson', 'NJ') ('fountainville', 'PA')
 ('huntingdonvalleypa', 'PA') ('washingtoncrossing', 'PA')
 ('linelexington', 'PA') ('newtownsquare', 'PA') ('earthcity', 'MO')
 ('lumberton', 'NJ') ('lahaska', 'PA') ('woolwichtwp', 'NJ')
 ('washingtontownship', 'NJ') ('mountholly', 'NJ')
 ('deptfordtownship', 'NJ') ('willingborotownship', 'NJ')
 ('columbus', 'NJ') ('hainesport', 'NJ') ('mendenhall', 'PA')
 ('dresher', 'PA') ('douglassville', 'PA') ('mapleshadetownship', 'NJ')
 ('secane', 'PA') ('lafayettehill', 'PA') ('elverson', 'PA')
 ('willingboro', 'NJ') ('woolwichtownship', 'NJ') ('hilltown', 'PA')
 ('westampton', 'NJ') ('shamong', 'NJ') ('mountroyal', 'NJ')
 ('jamison', 'PA') 

In [19]:
from collections import Counter
missing_counts = dict(Counter(missing_locs).most_common())

In [20]:
missing_counts

{('warrington', 'PA'): 203,
 ('newtownsquare', 'PA'): 177,
 ('mapleshade', 'NJ'): 148,
 ('cinnaminson', 'NJ'): 120,
 ('mountholly', 'NJ'): 120,
 ('delran', 'NJ'): 102,
 ('limerick', 'PA'): 78,
 ('willingboro', 'NJ'): 63,
 ('lumberton', 'NJ'): 62,
 ('lafayettehill', 'PA'): 58,
 ('hainesport', 'NJ'): 52,
 ('lahaska', 'PA'): 50,
 ('dresher', 'PA'): 49,
 ('jamison', 'PA'): 38,
 ('westampton', 'NJ'): 36,
 ('deptfordtownship', 'NJ'): 35,
 ('pennsaukentownship', 'NJ'): 33,
 ('columbus', 'NJ'): 32,
 ('buckingham', 'PA'): 28,
 ('riverside', 'NJ'): 22,
 ('washingtoncrossing', 'PA'): 22,
 ('earthcity', 'MO'): 22,
 ('mapleshadetownship', 'NJ'): 21,
 ('washingtontownship', 'NJ'): 18,
 ('christiana', 'DE'): 18,
 ('secane', 'PA'): 17,
 ('tabernacle', 'NJ'): 15,
 ('pilesgrove', 'NJ'): 14,
 ('perkiomenville', 'PA'): 13,
 ('shamong', 'NJ'): 13,
 ('wallingford', 'PA'): 13,
 ('greenfield', 'IN'): 12,
 ('lawrenceville', 'NJ'): 11,
 ('edgewaterpark', 'NJ'): 11,
 ('clarksboro', 'NJ'): 11,
 ('greenlane', 'PA'

In [ ]:
cities, states = zip(*missing_counts.keys())

In [34]:
missing_counts_df = pd.DataFrame([(city, state, count) for (city, state), count in zip(missing_counts.keys(), missing_counts.values())])

In [11]:
print(f"Number of Tuples in Original: {df["tup"].shape[0]}")
print(f"Number of Unique Tuples in Original: {df["tup"].unique().shape[0]}")

print(f"Number of Tuples with place codes: {place_df.shape[0]}")
print(f"Number of Unique Tuples with Place Codes: {clean_df.shape[0]}")
print(f"{missing_locs.shape[0]} Missing Locations")

Number of Tuples in Original: 144767
Number of Unique Tuples in Original: 1176
Number of Tuples with place codes: 13957
Number of Unique Tuples with Place Codes: 1306
371 Missing Locations


In [36]:
missing_counts_df.to_csv("missing_counts.csv", index=False)